# Python Cookbook 3.0 中文版

## 第一章：数据结构和算法

### 1.1 解压序列赋值给多个变量

任何的序列（或者是可迭代对象）可以通过一个简单的赋值语句解压并赋值给多
个变量。唯一的前提就是变量的数量必须跟序列元素的数量是一样的。

In [36]:
p = (4, 5)
x, y = p
print(x, y)

4 5


In [37]:
data = ['ACME', 50, 91.1, (2021, 10, 10)]
name, shares, price, date = data
print(name, shares, price, date)

ACME 50 91.1 (2021, 10, 10)


In [38]:
name, shares, price, (year, mon, day) = data
print(year, day)

2021 10


如果变量个数和序列元素的个数不匹配，会产生一个异常。

In [39]:
p = (x, y)
x, y, z = p

ValueError: not enough values to unpack (expected 3, got 2)

这种解压赋值可以用在任何可迭代对象上面，而不仅仅是列表或者元组。包括字符串，文件对象，迭代器和生成器。

In [ ]:
s = "Hello"
a, b, c, d, e = s
print(a,b,c,d,e)

如果只想解压一部分，丢弃掉其他的值，可以使用任意的变量名去占位，一般使用 `_` 来表示。

In [ ]:
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
_, shares, price, _ = data
print(shares, price)

### 1.2 解压可迭代对象赋值给多个变量

如果一个可迭代对象的元素个数超过变量个数时，会抛出一个 `ValueError` ，可以使用星号表达式来解决这个问题，例如想要统计一下家庭作业的平均成绩，但是要排除掉第一个和最后一个分数：

In [ ]:
def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

星号表达式可以用在列表的开始部分，也可以用在列表的末尾。星号表达式在迭代元素为可变长元组的序列时也是很有用的。

In [ ]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
    ]

def do_foo(x, y):
    print('foo', x, y)

def do_bar(s):
    print('bar', s)

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

星号解压语法在字符串操作的时候也会很有用，比如字符串的分割。

In [ ]:
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
print(uname, homedir, sh)

### 1.3 保留最后 N 个元素

## 第二章：字符串和文本

### 2.1 使用多个界定符分割字符串

string 对象的 split() 方法只适应于非常简单的字符串分割情形，它并不允许有多个分隔符或者是分隔符周围不确定的空格，当需要更加灵活的切割字符串的时候，最好使用 re.split() 方法：

In [ ]:
line = 'asdf fjdk; afed, fjek,asdf, foo'
import re
re.split(r'[\s;,]\s*', line)

如果需要保留分隔符，可以使用分组来实现：

In [ ]:
fields = re.split(r'(;|,|\s)\s*', line)
fields

In [ ]:
# 取值，[::2] 指的是隔一个索引取一个值，[1::2] 指的是从第一个值开始隔一个取一个
delemiters = fields[1::2]
delemiters

如果想保留分割字符串到结果列表中去，但仍然需要使用到括号来分组正则表达式的话，确保分组是非捕获分组，形如 (?:...)：

In [ ]:
re.split(r'(?:,|;|\s)\s*', line)

### 2.2 字符串开头或结尾匹配

指定文本模式去检查字符串的开头或者结尾，比如文件名的后缀、URL Scheme 等。

In [ ]:
filename = 'spam.txt'
filename.endswith('.txt')

In [ ]:
filename.startswith('file:')

In [ ]:
url = 'http://www.python.org'
url.startswith('http:')

如果想要匹配多种可能，需要将所有的匹配项放入到一个元组中，然后传给 `startswith()` 或者 `endswith()` 方法：

In [ ]:
filenames = [ 'Makefile', 'foo.c', 'bar.py', 'spam.c', 'spam.h' ]
[ name for name in filenames if name.endswith(('.c', '.h')) ]

In [ ]:
# all() 函数用于判断给定的可迭代参数 iterable 中的所有元素是否都为 TRUE，如果是返回 True，否则返回 False。
# any() 函数用于判断给定的可迭代参数 iterable 是否全部为 False，则返回 False，如果有一个为 True，则返回 True
any(name.endswith('.py') for name in filenames)

In [ ]:
# 一个实例，判断文件夹中是否存在指定的文件类型
import os
if any(name.endswith(('.c', '.h')) for name in os.listdir(dirname)): ...

### 2.3 用 Shell 通配符匹配字符串

`fnmatch()` 函数匹配能力介于简单的字符串方法和强大的正则表达式之间，如果在数据处理操作中只需要简单的通配符就能完成的时候，这通常是一个比较合理的方案。此模块的主要作用是文件名称的匹配，并且匹配的模式使用的Unix shell风格。

In [ ]:
from fnmatch import fnmatch, fnmatchcase

In [ ]:
fnmatch('foo.txt', '*.txt')

In [ ]:
fnmatch('foo.txt', '?oo.txt')

In [ ]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

在处理非文件名的字符串的时候，也可以用这个模块。

### 2.4 字符串匹配和搜索

如果你想匹配的是字面字符串，那么你通常只需要调用基本字符串方法就行，比如 str.find() , str.endswith() , str.startswith() 或者类似的方法。对于复杂的匹配要使用正则表达式和 re 模块。

In [ ]:
# match() 总是从字符串开始去匹配，如果想查找字符串任意部分的模式出现位置，使用 findall() 方法去代替
import re
text1 = '11/27/2012  3/13/2013'
text2 = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
datepat = re.compile(r'\d+/\d+/\d+')
r = re.match(datepat, text1)
print(r.group())
r = re.findall(datepat, text2)
print(r)

定义正则表达式时，通常会使用括号来捕获分组，这样可以把每个分组的内容提取出来，使得后面的处理更加简单：

In [ ]:
import re
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')
text1 = '11/27/2012  3/13/2013'
r = re.match(datepat, text1)
print(r.groups())
month, day, year = r.groups()
print(month, day, year)

In [ ]:
text2 = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
r = re.findall(datepat, text2)
print(r)

In [ ]:
# findall() 返回的是列表，如果返回的数据比较多，可以使用 finditer() 返回迭代器
r = re.finditer(datepat, text2)
print([_.groups() for _ in r])

### 2.5 字符串搜索和替换

In [ ]:
points = [ (1, 2), (3, 4), (5, 6), (7, 8) ]

import math
from functools import partial

def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2-x1, y2-y1)


In [ ]:
pt = (4, 3)

In [ ]:
class A:
    def spam(self, x):
        print('in class A', x)
    def foo(self):
        print('in class A')
        
class B1:
    def __init__(self):
        self._a = A()
        
    def spam(self, x):
        return self._a.spam(x)
    def foo(self):
        return self._a.foo()
    def bar(self):
        pass

In [ ]:
b = B1()

In [ ]:
b.spam(111)

In [ ]:
class B2:
    def __init__(self):
        self._a = A()
    def __getattr__(self, name):
        return getattr(self._a, name)

In [ ]:
b2 = B2()

In [ ]:
b2.spam(222)

### 代理模式

In [ ]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
    def __getattr__(self, name):
        print('Getattr:',name)
        return getattr(self._obj, name)
    
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            print('setattr:', name, value)
            setattr(self._obj, name, value)
            
    def __delattr__(self, name):
        if name.startswith('_'):
            super.__delattr__(name)
        else:
            print('Delattr"', name)
            delattr(self._obj, name)

In [ ]:
class Spam:
    def __init__(self, x):
        self.x = x
    def bar(self, y):
        print('Spam.bar:', slef.x, y)
        

In [ ]:
s = Spam(2)

In [ ]:
s.x

In [ ]:
s1 = Proxy(s)

In [ ]:
s.x = 20
s.x

In [ ]:
s1.x

### 上下文管理

In [ ]:
class A():
    def __init__(self, ip=None, username=None, password=None, port=22, test=111):
        self.ip = ip
        self.username = username
        self.password = password
        self.port = port
        self.conn = None
    
    def __enter__(self):
        if self.ip:
            print(f"正在连接 {self.ip} ...")
            return self.conn
        else:
            raise "信息不全，无法连接"
        
# __exit__() 方法的第三个参数包含了异常类型、异常值和追溯信息 (如果有的话)
#     def __exit__(self, exc_type, exc_value, tracebalk):
    def __exit__(self, *args):
#         msg = {"exc_type":exc_type,
#                "exc_value":exc_value,
#                "tracebalk":tracebalk}
        self.conn = "连接关闭"
        print(self.conn)
#         print(msg)
#        如果返回内容为 True，就不会报出错误信息
#         return args

In [ ]:
a = A(ip="1.1.1.1")
with a as aa:
    print("执行命令 xxxxx")
    raise Exception("错误执行")

In [ ]:
class A:
    def spam(self):
        print("A.spam")
        super().spam()

### 添加类属性 property

In [ ]:
class Person:
    def __init__(self, first_name) -> None:
        self.first_name = first_name
    
    # getter 方法
    @property
    def first_name(self):
        return self._first_name
    # setter 方法
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError("必须是一个字符串")
        self._first_name = value
    # deleter 方法
    @first_name.deleter
    def first_name(self):
        raise AttributeError("不能删除这个属性")

上面代码中有三个相关联的方法，三个方法的名字必须一样：
- 第一个方法是一个 getter 函数，使 first_name 方法变成了一个属性
- 其他两个方法给 first_name 属性添加了 setter 和 deleter 函数
- 只有在 first_name 属性被创建后，后面的两个装饰器才能被定义使用

In [ ]:
a = Person("张三")
a.first_na

In [ ]:
a.first_name = 18

In [ ]:
del a.first_name

### 简化数据结构的初始化

当你需要使用大量很小的数据结构类的时候，相比手工一个个定义 `__init__()` 方法，可以在基类中写一个公用的 `__init__()` 函数，子类继承父类实现，使用这种方式可以大大简化代码：

In [42]:
class Structure1:
    _fields = []

    def __init__(self, *args) -> None:
        if len(args) != len(self._fields):
            raise TypeError(f"需要传入 {len(self._fields)} 个参数。")
        for name, value in zip(self._fields, args):
            setattr(self, name, value)

In [41]:
# 然后定义数据结构相关的类，继承基类
class Person(Structure1):
    _fields = ["name", "age", "sex"]

a = Person("张三", 18, "男")
a.__dict__

{'name': '张三', 'age': 18, 'sex': '男'}

### 定义接口或者抽象基类

使用 `abc` 模块定义一个抽象基类，并且通过执行类型检查来确保子类实现某些特定的方法。

In [ ]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbyte=-1):
        pass
    @abstractmethod
    def write(self, data):
        pass
# 抽象类不能被实例化，它的目的是让别的类继承它并实现特定的抽象方法
a = IStream()

In [45]:
# 抽象基类还用来检查某些类是否为特定的类型
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError("类型错误，应该是 IStream")
    pass

# 除了继承，还能通过注册的方式来让某个类实现抽象基类
import io
IStream.register(io.IOBase)

io.IOBase

## 元编程

### 装饰器

In [ ]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"执行共花费时间 {end - start}s")
        return result
    return wrapper


In [51]:
@timethis
def count(n):
    while n > 0:
        n -= 1
count(10000000)

执行共花费时间 0.7779171466827393s


**装饰器执行的时候，被装饰函数会被当做第一个参数直接传递给 timethis 函数，所以，timethis 中的第一个参数就是被包装函数本身。**

In [12]:
def count1(n):
    while n > 0:
        n -= 1
count1 = timethis(count1)
count1(10000000)

执行共花费时间 0.7739660739898682s


### 创建装饰器时保留函数元信息

任何时候你定义装饰器的时候，都应该使用 functools 库中的 @wraps 装饰器来注解底层包装函数，来保留函数的元信息，例如名字、文档字符串、注解、参数签名等。

In [5]:
# 添加函数的文档字符串
import time
from functools import wraps

def timethis(func):
    '''
    一个用来统计运行时间的装饰器
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"执行共花费时间 {end - start}s")
        return result
    return wrapper

@timethis
def count(n):
    '''一个用来倒数计数的函数'''
    while n > 0:
        n -= 1
    print("执行完成")

In [79]:
# 如果注释掉上面代码中的 @wraps(func) ，那么函数的名字就是 'wrapper'，原来函数的名字就被装饰器替换掉了。
count.__name__

'count'

### 解除装饰器

一个装饰器已经作用在一个函数上了，如果想要取消装饰器，只执行原来的函数。

假设装饰器是通过 `@wraps` 来实现的，那么可以通过访问 `__wrapped__` 属性来访问到原来的函数。

In [80]:
count(100000)

执行完成
执行共花费时间 0.012969732284545898s


In [81]:
count.__wrapped__(100000)

执行完成


### 带参数的装饰器

In [13]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    def decorate(func):
        logging.basicConfig(level=level)
        logname = name if name else func.__module__
        logger = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            logger.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [95]:
@logged(level=logging.DEBUG,name="加法函数", message="加法")
def add(x, y):
    return x + y

add(1, 2)

DEBUG:加法函数:加法


3

使用带参数的装饰器时，运行效果等同于：`logged(level=logging.DEBUG,name="加法函数", message="加法")(spam)`
- 先带参数运行 logged，返回值是内层的装饰器：`decorate`
- 内层装饰器再把被装饰器函数传入并执行：`decorate(spam)`

带参数的装饰器必须要传递参数

### 可以让用户自定义属性的装饰器

写一个装饰器来包装一个函数，并且允许用户提供参数在运行时控制装饰器行为，使用 nonlocal 修改内部变量，并将一个访问函数作为一个属性赋值给包装函数。

In [24]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        print("执行了 partial")
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    print("执行了 setattr ")
    return func

def logged(level, name=None, message=None):
    def decorate(func):
        logging.basicConfig(level=level)
        logname = name if name else func.__module__
        logger = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            logger.log(level, logmsg)
            return func(*args, **kwargs)
        
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
        return wrapper
    return decorate


In [25]:
# @timethis
@logged(logging.DEBUG)
def add(x, y):
    return x + y

add(2,3)

DEBUG:__main__:add


执行了 partial
执行了 setattr 


5

In [105]:
add.set_level(logging.WARNING)

In [106]:
add(2,3)

5

### 带可选参数的装饰器

写一个装饰器，既可以不传参使用，比如 `@decorator`，也可以传递可选参数使用，比如 `decorator(x, y, z)`。

In [40]:
# 对上一节中的带参数的装饰器进行修改
from functools import partial, wraps
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        print(f"partial 运行了")
        return partial(logged, level=level, name=name, message=message)
    print(f"partial 没有运行，传入的函数是 {func.__name__}")
    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__

    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        print(f"{func.__name__} 执行了")
        return func(*args, **kwargs)
    return wrapper


In [42]:
@logged
def add(x, y):
    return x + y

add(1, 3)

DEBUG:__main__:add


partial 没有运行，传入的函数是 add
add 执行了


4

In [43]:
@logged(level=logging.WARNING,message="example")
def foo():
    print("foo")

foo()

partial 运行了
partial 没有运行，传入的函数是 foo
foo 执行了
foo


In [44]:
import webbrowser

webbrowser.open("www.baidu.com")

True